In [6]:
import os
import pandas as pd
import donedeal as dd

pd.options.display.max_columns = None

data = []
for file in filter(lambda file: file.endswith('.pkl'),os.listdir('.')):
    if file == 'processed.pkl':
        continue
    data.append(pd.read_pickle(file))
data = pd.concat(data,ignore_index=True)

ds = data.copy().drop(columns = 'ScrapeDate countyTown friendlyUrl header description'.split())
ds = ds.apply(lambda row:row.apply(lambda x: x if x else None),axis=1)
ds = ds.dropna()

ds['seats'] = ds.seats.astype('int')
ds['numDoors'] = ds.numDoors.astype('int')
ds['owners'] = ds.owners.astype('int')
ds['roadTax'] = ds.roadTax.astype('int')
ds['year'] = ds.year.astype('int')

ds['NCTExpiry'] = ds.NCTExpiry.apply(dd.data_cleaning.parse_NCTExpiry)
ds['engine'] = ds.engine.apply(dd.data_cleaning.parse_engine)
ds['enginePower'] = ds.enginePower.apply(dd.data_cleaning.parse_enginePower)
ds['acceleration'] = ds.acceleration.apply(dd.data_cleaning.parse_acceleration)

ds.apply(lambda row: row.apply(lambda x: type(x).__name__).value_counts().to_dict(),axis=0)
types = ds.apply(lambda row: row.apply(lambda x: type(x).__name__).iloc[0],axis=0).sort_values().to_dict()
ds = ds[types.keys()]

inverse_types = dict()
for col,dtype in types.items():
    if dtype not in inverse_types:
        inverse_types[dtype] = []
    inverse_types[dtype].append(col)

ds.reset_index(drop=True,inplace=True)
exploded = pd.concat([ds]+[dd.data_cleaning.encode_strings(ds[col]) for col in inverse_types['str']],axis=1).drop(columns=inverse_types['str'])
exploded.to_pickle('processed.pkl')

county
sellerType
currency
make
country
transmission
trim
fuelType
model
bodyType
colour
